---
layout: post
title: "CNN에 관하여"
author: "Chanjun Kim"
categories: Data분석
tags: [DeepLearning, DL, CNN, 이미지, 영상]
image: 03_entropy_gini.png
---

## **학습목적**
이미지 데이터를 처리하고 학습하는 딥러닝 모델 중 CNN을 공부하고 오픈 이미지 데이터를 가지고 한 스텝 한 스텝 따라가본다.<br>

## **CNN이란?**
Convolutional Neural Network의 약자로 주로 이미지나 영상 데이터를 처리하기 위한 딥러닝 알고리즘 중 하나이다. <br>
Flatten한 데이터를 처리하는 Fully Connected Neural Network와 다르게 이미지의 공간 정보 데이터를 유지하여 이미지를 인식할 수 있도록 만든다.<br>
이를 위해서 Filter, Pooling, Padding, Stride 등의 개념들이 사용되며, 천천히 알아가보도록 하겠습니다.

> 출처/참고자료 : http://taewan.kim/post/cnn/

간단한 이미지를 참고해서 가져오겠습니다. 출처는 [http://taewan.kim/post/cnn/](http://taewan.kim/post/cnn/) 입니다.<br>
사람은 간단하게 2라고 보이는 것을 알 수 있지만, 컴퓨터 혹은 딥러닝으로 학습시키기에는 이렇게만 정보가 있어서는 안됩니다.<br>
<img src = "https://blog.kakaocdn.net/dn/cJXmju/btqvGBJVMH0/WdMzvcKhEiMvaIAZuoYQfk/img.png" alt = "Oops" width = "300"/>
<br>

그래서 위와 같이 숫자로 이루어진 바둑판 모양으로 변환을 하게됩니다.<br>
<img src = "https://blog.kakaocdn.net/dn/bBoEcG/btqvGDucYov/thqHXwKvveVJmsbXJkHns0/img.png" alt = "Oops" width = "300"/>
<br>

그리고 각 위치별 특성을 보기 위하여 filter라는 어떠한 판을 만들어 그것을 곱하여 특성으로 사용되게 됩니다.<br>
<img src = "https://blog.kakaocdn.net/dn/Qa6UF/btqvFXmb9XY/nAMjiM4QwUgZ1K7egHWvsk/img.png" alt = "Oops" width = "600"/>

참고 그림이 잠깐 달라지지만, 결국 아래와 같은 Feature map을 만들 수 있습니다.<br>
여기서는 [[1, 0, 1], [0, 1, 0], [1, 0, 1]] 이렇게 생긴 filter를 사용했네요.<br>
<img src = "http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif" alt = "Oops" width = "600"/>
<br>

Filter에 대한 값은 [Keras layer initializers](https://keras.io/api/layers/initializers/)에서 확인하실 수 있습니다.<br>
다양하게 초기화시킬 수 있지만, 이 파라미터를 크게 바꾸는 경우나 바꿔서 크게 성능이 좋아지는 경우는 아직 확인하지 못했습니다.<br>
<br>

이러한 Convolution과 filter의 개념을 사용하여 아래와 같은 그림으로 CNN이 이루어지게 됩니다.<br>
<img src = "https://blog.kakaocdn.net/dn/NWDaW/btqvHgZ2d0Z/sEV8R4ANc3HltM2eKvmhvk/img.jpg" alt = "Oops" width = "800"/>

> 출처/참고자료 : http://taewan.kim/post/cnn/<br>
> 출처/참고자료 : https://youngq.tistory.com/40

그럼 이제 위에서 나오는 channel, Pooling, padding, Stride 등의 용어에 대해서 정리해보겠습니다.

- Channel
    - 채널은 이미지의 차원?이라고 생각하시면 될 것 같습니다. 흑백(Grey scale) 이미지에서는 1개의 채널을 갖지만 보통 우리가 많이 보는 색상이 있는 이미지라면 RGB라는 총 3개의 채널을 갖게 됩니다. ~~(투명도(alpha) 채널이 추가되는 경우도 있다고 합니다.)~~
    - 처음 이미지 채널을 시작으로 Convolution Layer의 채널 수를 늘려가면서 Hidden 층을 깊게 만들어 계산하게 됩니다.
    - Pooling 과정을 거치며 이미지 사이즈는 줄어들지만, Channel 수가 늘어나면서 연산량이 늘어나게 됩니다.
- Filter
    - Filter는 위에서 말씀드렸듯이 어떠한 Channel에 곱해지는 하나의 랜덤한 판입니다. 연산을 하는 과정에서 Back propagation(역전파)를 통하여 이 filter의 값을 업데이트하면서 최적해를 찾아가게 됩니다.
- Pooling
    - Pooling은 계속 연산량만 늘어나는 것을 방지하기 위해서 feature map 상에서 원하는 정보를 추출할 수 있도록 해주는 장치입니다.
    - 보통 feature map 상에서 가장 두들어진 부분을 뽑아내기 위하여 max pooling이라는 기법을 활용하고, 상황에 따라 전체적인 부분을 보기 위해서 average pooling과 같은 기법도 사용한다고 합니다. ~~보통 max pooling을 사용합니다.~~
    
---

### **이 외에도 loss function, Optimizer, Activate function 등 아직 배워야할 것이 많기 때문에 슬 코드로 시작해보도록 하겠습니다.**

In [10]:
import os
import sys
import warnings
from glob import glob

import numpy as np
import pandas as pd

import tensorflow as tf

# import cv2

In [8]:
!python --version

Python 3.6.13 :: Anaconda, Inc.


In [9]:
tf.__version__

'2.5.0'

오픈 데이터셋을 제공해주는 AI HUB에서 한국 음식 데이터를 받아서 실습을 해보겠습니다. [https://aihub.or.kr/aidata/13594](https://aihub.or.kr/aidata/13594) <br>
많은 종류가 있지만 제 취향 상 구이 카테고리를 분류해보도록 하겠습니다.

In [29]:
file_list = pd.DataFrame(glob("data/kfood/구이/*/*"), columns = ["dir_name"])
file_list

,dir_name
0,data/kfood/구이\갈비구이\crop_area.properties
1,data/kfood/구이\갈비구이\Img_000_0000.jpg
2,data/kfood/구이\갈비구이\Img_000_0001.jpg
3,data/kfood/구이\갈비구이\Img_000_0002.jpg
4,data/kfood/구이\갈비구이\Img_000_0003.jpg
...,...
14024,data/kfood/구이\훈제오리\Img_013_0997.jpg
14025,data/kfood/구이\훈제오리\Img_013_0998.jpg
14026,data/kfood/구이\훈제오리\Img_013_0999.jpg
14027,data/kfood/구이\훈제오리\Img_013_1000.jpg


In [39]:
file_list.dir_name = file_list.dir_name.str.replace(pat = "\\", repl = "/")
file_list["category"] = file_list.dir_name.str.replace(pat = ".*/.*/.*/(.*)/(.*)(\.*)", repl = "\\1")
file_list["file_name"] = file_list.dir_name.str.replace(pat = ".*/.*/.*/(.*)/(.*)(\.*)", repl = "\\2")
file_list["ext_name"] = file_list.dir_name.str.replace(pat = ".*/.*/.*/(.*)/(.*)\.(.*)", repl = "\\3")
file_list = file_list[file_list.ext_name.isin(["JPG", "jpg"])]
file_list

,dir_name,category,file_name,ext_name
1,data/kfood/구이/갈비구이/Img_000_0000.jpg,갈비구이,Img_000_0000.jpg,jpg
2,data/kfood/구이/갈비구이/Img_000_0001.jpg,갈비구이,Img_000_0001.jpg,jpg
3,data/kfood/구이/갈비구이/Img_000_0002.jpg,갈비구이,Img_000_0002.jpg,jpg
4,data/kfood/구이/갈비구이/Img_000_0003.jpg,갈비구이,Img_000_0003.jpg,jpg
5,data/kfood/구이/갈비구이/Img_000_0004.jpg,갈비구이,Img_000_0004.jpg,jpg
...,...,...,...,...
14023,data/kfood/구이/훈제오리/Img_013_0996.jpg,훈제오리,Img_013_0996.jpg,jpg
14024,data/kfood/구이/훈제오리/Img_013_0997.jpg,훈제오리,Img_013_0997.jpg,jpg
14025,data/kfood/구이/훈제오리/Img_013_0998.jpg,훈제오리,Img_013_0998.jpg,jpg
14026,data/kfood/구이/훈제오리/Img_013_0999.jpg,훈제오리,Img_013_0999.jpg,jpg


파일들은 거의 1000개씩 고루고루 있으므로 이번에도 제 취향대로 [삼겹살, 갈비구이, 곱창구이, 장어구이, 불고기, 조개구이] 정도를 선택하도록 하겠습니다.

In [41]:
file_list.reset_index(drop = True, inplace = True)
file_list.groupby("category").file_name.count()

category
갈비구이      998
갈치구이      998
고등어구이    1000
곱창구이     1000
닭갈비       996
더덕구이     1000
떡갈비       997
불고기       997
삼겹살       992
장어구이      997
조개구이     1000
조기구이     1000
황태구이      997
훈제오리      992
Name: file_name, dtype: int64

In [43]:
file_list = file_list[file_list.category.isin(["삼겹살", "갈비구이", "곱창구이", "장어구이", "불고기", "조개구이"])]
print(len(file_list))
print(file_list.category.unique())

5984
['갈비구이' '곱창구이' '불고기' '삼겹살' '장어구이' '조개구이']


---

참고 자료 :
- https://youngq.tistory.com/40

In [ ]:
file_list = pd.DataFrame(glob("data/kfood/구이/*/*"), columns = "file_name")